In [2]:
import pandas as pd
import requests
from fuzzywuzzy import fuzz

In [3]:
kpop_bbh100 = pd.read_csv('Kpop on BBH100.csv')
kpop_bbh100.dropna(how='all', inplace=True)
kpop_bbh100.dropna(axis=1, how='all', inplace=True)
kpop_bbh100.drop(labels=13, inplace=True) #drop baby shark
kpop_bbh100.head()

,Chart date[16],Artist,Song/Album Title,Peak position,Weeks on Chart
0,2009-10-31,Wonder Girls,"""Nobody (English release)""[17]",76,1
1,2012-09-22,Psy[18],"""Gangnam Style""[19][20]",2 (Peaked at #2 on 2012-10-06 for 7 weeks),31
2,2013-04-27,Psy,"""Gentleman""",5,15
3,2014-06-28,Psy feat. Snoop Dogg,"""Hangover""",26,1
4,2015-12-19,Psy feat. CL[21],"""Daddy""",97,1


In [4]:
##standardize and process chart data

#remove wiki citations, standardize whitespace, remove quotation marks
#regex patterns and replacement strings
regex_pats = [[r"(\[\d+\])", ""], [r"(\s)", " "], [r"(\")", ""]]
for pat in regex_pats:
    kpop_bbh100["Artist"] = kpop_bbh100["Artist"].str.replace(pat[0], pat[1])
    kpop_bbh100["Song/Album Title"] = kpop_bbh100["Song/Album Title"].str.replace(pat[0], pat[1])
    kpop_bbh100["Peak position"] = kpop_bbh100["Peak position"].str.replace(pat[0], pat[1])

#coerce strings to lowercase
kpop_bbh100["Artist"] = kpop_bbh100["Artist"].str.lower()
kpop_bbh100["Song/Album Title"] = kpop_bbh100["Song/Album Title"].str.lower()

kpop_bbh100

,Chart date[16],Artist,Song/Album Title,Peak position,Weeks on Chart
0,2009-10-31,wonder girls,nobody (english release),76,1
1,2012-09-22,psy,gangnam style,2 (Peaked at #2 on 2012-10-06 for 7 weeks),31
2,2013-04-27,psy,gentleman,5,15
3,2014-06-28,psy feat. snoop dogg,hangover,26,1
4,2015-12-19,psy feat. cl,daddy,97,1
5,2016-10-22,cl,lifted,94,1
6,2017-10-07,bts,dna,67,4
7,2017-12-16,bts feat. desiigner,mic drop (steve aoki remix),28,10
8,2018-06-02,bts,fake love,10,6
9,2018-06-30,blackpink,ddu-du ddu-du,55,1


In [5]:
####genius
client_id = 'LOe0w-dmJDdzfnsVQ6r6j3Zp1EakdMoj9YqaOBcAF13P3vOfXB6QmiLKl2Qyc_ue'
client_secret = 'JxZQkmfZIvlkjpE--HFPsI7zV3Freyt4WWMDIEnQEtb6BJGi1ZjTr4qK_vOsyawlfKoOcw4li8q_M0X48ZGU4A'
client_access_token = 'KQ5oZoRnw1ea0YwPjOeDFacmn37UlUDjp8DGKlj7pfgDsIXJYfOz00skKmR8hqhR'

artist = kpop_bbh100["Artist"][1]
this_parameters = {
            "q": artist,
            "access_token": client_access_token
        }
this_result = requests.get(url, params=this_parameters)

In [6]:
url = 'https://api.genius.com/search?'

for artist in kpop_bbh100["Artist"].unique():
    print(artist)

wonder girls
psy
psy feat. snoop dogg
psy feat. cl
cl
bts
bts feat. desiigner
blackpink
bts feat. nicki minaj
dua lipa & blackpink
steve aoki feat. bts
bts feat. halsey
bts  bts feat. lauv
j-hope feat. becky g
suga (agust d)
lady gaga and blackpink
jawsh 685, jason derulo & bts
blackpink and selena gomez


In [116]:
hits = this_result.json()["response"]["hits"]
potential_matches = []

for h in hits:
    page = h["result"]
    song_title = page["full_title"].lower()
    title_ratio = fuzz.partial_ratio(song_title, title)

    artist_name = page["primary_artist"]["name"].lower()
    artist_ratio = fuzz.partial_ratio(artist_name, artist)

    if title_ratio >= 90 and artist_ratio >= 90:
        potential_matches.append(page)

potential_matches

[{'annotation_count': 4,
  'api_path': '/songs/123994',
  'full_title': 'Gangnam Style (Diplo Remix) by\xa0PSY (Ft.\xa02\xa0Chainz, Diplo & Tyga)',
  'header_image_thumbnail_url': 'https://images.genius.com/a1e271027860efffd79001b7d0e1ff95.300x306x1.jpg',
  'header_image_url': 'https://images.genius.com/a1e271027860efffd79001b7d0e1ff95.608x620x1.jpg',
  'id': 123994,
  'lyrics_owner_id': 69536,
  'lyrics_state': 'complete',
  'path': '/Psy-gangnam-style-diplo-remix-lyrics',
  'pyongs_count': 2,
  'song_art_image_thumbnail_url': 'https://images.genius.com/a1e271027860efffd79001b7d0e1ff95.300x306x1.jpg',
  'song_art_image_url': 'https://images.genius.com/a1e271027860efffd79001b7d0e1ff95.608x620x1.jpg',
  'stats': {'unreviewed_annotations': 2, 'hot': False, 'pageviews': 10615},
  'title': 'Gangnam Style (Diplo Remix)',
  'title_with_featured': 'Gangnam Style (Diplo Remix) (Ft.\xa02\xa0Chainz, Diplo & Tyga)',
  'url': 'https://genius.com/Psy-gangnam-style-diplo-remix-lyrics',
  'primary_ar

In [7]:
###search for artist names to get ids - identify via partial ratio match
####use artists ids to get 50 most popular songs
#####identify songs in bbh100 usign partial ratio match
##store song urls
##scrape lyrics

##then need to search for english translations??

#OR JUST do it manually bc genius search is jank :p

In [3]:
type(open("Manual Lyrics/Daddy.txt", "r").read())

str